In [2]:
import torch
import torchvision

import os
import pathlib
import itertools

from tqdm import tqdm

from torchsummary import summary

import src.utils.tensor
import src.utils.module
import src.utils.comps

import src.comps.backbones_trf
import src.comps.backbones_trf_pyramid

---

In [3]:
model = src.comps.backbones_trf.GCVitTinyBackbone(img_size=224)

/home-net/gortega/fashion_retrieval/.venv/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
summary(model, input_size=(3, 224, 224), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           1,792
         LayerNorm-2         [-1, 112, 112, 64]             128
            Conv2d-3         [-1, 64, 112, 112]             576
              GELU-4         [-1, 64, 112, 112]               0
 AdaptiveAvgPool2d-5             [-1, 64, 1, 1]               0
            Linear-6                   [-1, 16]           1,024
              GELU-7                   [-1, 16]               0
            Linear-8                   [-1, 64]           1,024
           Sigmoid-9                   [-1, 64]               0
               SE-10         [-1, 64, 112, 112]               0
           Conv2d-11         [-1, 64, 112, 112]           4,096
           Conv2d-12           [-1, 64, 56, 56]          36,864
        LayerNorm-13           [-1, 56, 56, 64]             128
       ReduceSize-14           [-1, 56,

In [5]:
input_tensor = torch.rand(1, 3, 224, 224)
output_tensor = model(input_tensor)
src.utils.tensor.print_tensor_info(output_tensor)

shape:  torch.Size([1, 512, 7, 7])
dtype:  torch.float32
device:  cpu
mem:  98.07 KiB


---

In [6]:
model = src.comps.backbones_trf_pyramid.GCVitTinyMultilevelBackbone(img_size=224)

In [7]:
summary(model, input_size=(3, 224, 224), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           1,792
         LayerNorm-2         [-1, 112, 112, 64]             128
            Conv2d-3         [-1, 64, 112, 112]             576
              GELU-4         [-1, 64, 112, 112]               0
 AdaptiveAvgPool2d-5             [-1, 64, 1, 1]               0
            Linear-6                   [-1, 16]           1,024
              GELU-7                   [-1, 16]               0
            Linear-8                   [-1, 64]           1,024
           Sigmoid-9                   [-1, 64]               0
               SE-10         [-1, 64, 112, 112]               0
           Conv2d-11         [-1, 64, 112, 112]           4,096
           Conv2d-12           [-1, 64, 56, 56]          36,864
        LayerNorm-13           [-1, 56, 56, 64]             128
       ReduceSize-14           [-1, 56,

In [8]:
input_tensor = torch.rand(1, 3, 224, 224)
output_tensors = model(input_tensor)

In [9]:
for idx, output_tensor in enumerate(output_tensors):
    src.utils.tensor.print_tensor_info(output_tensor, name="Level{:d}".format(idx+1))
    print()

name: Level1
shape:  torch.Size([1, 64, 56, 56])
dtype:  torch.float32
device:  cpu
mem:  784.07 KiB

name: Level2
shape:  torch.Size([1, 128, 28, 28])
dtype:  torch.float32
device:  cpu
mem:  392.07 KiB

name: Level3
shape:  torch.Size([1, 256, 14, 14])
dtype:  torch.float32
device:  cpu
mem:  196.07 KiB

name: Level4
shape:  torch.Size([1, 512, 7, 7])
dtype:  torch.float32
device:  cpu
mem:  98.07 KiB



In [10]:
src.utils.module.get_num_params(model)

27707950